# 聚类Clustering

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 聚类

聚类是一种无监督学习方法，用于将数据点分组成多个集群/**簇**（cluster），使得同一集群内的数据点相似度高，而不同集群间的数据点相似度低。

聚类的应用领域包括：

- 市场细分
    - 根据顾客的行为和喜好分组
    - 向同一分组的顾客推出有针对性的促销
- 推荐系统
    - 根据用户的属性/行为（如：类型、时长）推荐服务（如：电影、视频）
- 图像分割

簇的个数极大影响了分组的质量：

![](./img/簇.png)

## K-Means

`K-Means`是一种非常流行和简单的聚类算法，用于将数据划分为`K`个不同的簇（clusters），其中每个数据点属于距离自己最近的中心（centroid）的簇。

算法步骤：

1. 初始化
    - 确定簇的数量`K`
    - 随机选择`K`个数据点作为初始中心

Repeat

2. 分配
    - 将每个数据点分配给最近的簇

3. 更新
    - 重新计算每个簇的中心，通常是簇内所有点的平均位置
    
Until 中心点不再变化

https://shabal.in/visuals/kmeans/6.html

优点：

- 实现简单、容易理解
- 计算效率高，适合大数据集
- K-Means一定会**收敛**

缺点：

- `K`值的选择对聚类的结果有很大影响
- 对初始中心的选择敏感
- 对噪声和异常值敏感

![](./img/KMeans限制.png)

https://shabal.in/visuals/kmeans/1.html

【Quiz】当$ K \ge 2 $时，`K-Means`在最差情况下会有多少个空簇？（B）

A. $ 0 $

B. $ K - 1 $

C. $ K $

D. $ K + 1 $

`houses.csv`记录了某地房屋的信息，根据房屋的位置进行聚类分析。

In [ ]:
df = pd.read_csv('./data/houses.csv')
df.head()

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(df['Latitude'], df['Longitude'])
plt.xlabel('纬度')
plt.ylabel('经度')
plt.title('房屋分布图')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
X = df[['Longitude', 'Latitude']]
X.head()

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(X)

In [ ]:
df['Cluster'] = kmeans.labels_
df.head()

In [ ]:
plt.scatter(df['Longitude'], df['Latitude'], c=df['Cluster'])
plt.xlabel('纬度')
plt.ylabel('经度')
plt.title('House Clusters')
plt.show()

## 鸢尾花

鸢尾花`iris.data`记录了`3`种种类（`Iris-setosa`、`Iris-versicolor`、`Iris-virginica`）的鸢尾花特征：

- 花萼Sepal的长度和宽度
- 花瓣Petal的长度和宽度

鸢尾花数据集是统计学家R. A. Fisher在20世纪中期发布的，被公认为数据挖掘最著名的数据集。

![](./img/鸢尾花.jpg)

In [ ]:
df = pd.read_csv('data/iris.data', header=None)
df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class']
df.head()

通过绘制特征项的散点图矩阵，可以观察到每两种特征的关系。

In [ ]:
pd.plotting.scatter_matrix(df, diagonal='hist')
plt.show()

可以看出，虽然有3种类型的鸢尾花，但是大部分特征值明显聚为2簇，这是因为`Iris-versicolor`、`Iris-virginica`区分度不是特别显著。

使用`KMeans`分为3簇：

In [ ]:
X = df[['sepal length', 'sepal width', 'petal length', 'petal width']]
X.head()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)

In [ ]:
pd.plotting.scatter_matrix(df, c=kmeans.labels_, diagonal='hist')
plt.show()

## 性能评估

### 有分类标签的数据集

对于有分类标签的数据集，可以使用**兰德指数**（Adjusted Rand Index）评估聚类性能，它能够计算真实类别与聚类类别两种分布之间的相似性，取值范围为$ [0, 1] $。

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score

In [ ]:
df.loc[df['class'] == 'Iris-setosa', 'class'] = 0
df.loc[df['class'] == 'Iris-versicolor', 'class'] = 1
df.loc[df['class'] == 'Iris-virginica', 'class'] = 2

In [ ]:
df.head()

In [ ]:
adjusted_rand_score(df['class'], kmeans.labels_)

### 无分类标签的数据集

对于无分类标签的数据集，可以使用轮廓系数（Silhouette Coefficient）来度量聚类的质量，其取值范围为$ [-1, 1] $，轮廓系数越大表示聚类效果越好。

In [ ]:
from sklearn.metrics.cluster import silhouette_score

In [ ]:
silhouette_score(X, kmeans.labels_)

### 确定`k`值

如果数据集没有已知类别，可以尝试多个不同的`k`值，通过比较轮廓系数，选择最合适的`k`值。

In [ ]:
clusters = [2, 3, 4, 5, 6, 7, 8]
scores = []

In [ ]:
for k in range(2, 9):
    kmeans = KMeans(n_clusters=k).fit(X)
    scores.append(silhouette_score(X, kmeans.labels_))

In [ ]:
scores

In [ ]:
plt.plot(clusters, scores, '*-')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.show()

可以发现，当$ k = 2 $时，聚类的轮廓系数最大。